Algorithme de Dijkstra
======================

L'algorithme de Dijkstra permet de trouver le plus court chemin dans un graphe pondéré, à partir d'un sommet du graphe vers n'importe quel autre sommet.

Pour implémenter efficacement cet algorithme, nous allons utiliser un **tas mutable**, structure de données qui possède les primitives suivantes, indiquées avec leurs complexités :
    
* créer un tas mutable vide $O(1)$
* tester si le tas est vide $O(1)$
* entasser un élément muni de sa priorité $O(\log n)$
* extraire l'élément minimal $O(\log n)$
* modifier la priorité d'un élément $O(\log n)$

C'est cette dernière primitive qui fait qu'on appelle cette structure un tas **mutable**.

Nous allons dans ce notebook implémenter la structure sous forme d'une classe. C'est un choix d'implémentation, basé sur la volonté d'offrir une couche d'abstraction et une bonne séparation interface/implémentation.

## Choix de la représentation

Le tas va être implémenté par un arbre binaire complet stocké dans un tableau `t`, la racine étant `t[0]` et les enfants de `t[k]` étant `t[2*k+1]` et `t[2*k+2]`.

La spécificité d'un tas **mutable** est qu'il faut pouvoir modifier la priorité d'un élément du tas, 
* d'abord en retrouvant rapidement la position de l'élément dans le tas (il est hors de question de parcourir tout le tas)
* ensuite, après modification de la priorité, il faudra déplacer l'élément modifié dans le tas afin de garder la structure de tas.

La structure de données étant au service d'un algorithme, on va utiliser deux spécificités de l'algorithme de Dijkstra qui vont simplifier la programmation :
* pas besoin d'entasser des éléments, il suffira de créer un tas initial qui contient tous les sommets avec une priorité $\infty$,
* pour Dijkstra, la priorité d'un sommet ne peut que diminuer, donc un élément dont la priorité a été modifiée ne peut que remonter dans le tas.

De plus, on aura besoin ici :
* d'accéder à la priorité d'un sommet, c'est-à-dire sa distance à l'origine,
* de récupérer toutes les distances à l'origine après avoir vidé le tas.

On va utiliser **trois** structures de données dans notre classe `FileP` :
* le tas à proprement parler, qui ne contiendra pas les priorités mais les sommets (leurs noms ou leurs numéros)
* un dictionnaire dont les clés seront les sommets et dont les valeurs seront les priorités associées
* un dictionnaire dont les clés seront les sommets et dont les valeurs seront les positions des sommets dans le tas

In [1]:
class FileP:
    def __init__(self,sommets):
        from math import inf
        self.tas=[s for s in sommets]
        self.prio={s:inf for s in sommets}
        self.pos={s:i for i,s in enumerate(sommets)}
    def vide(self):
        return self.tas==[]
    def extraire(self):
        res=self.tas[0]
        del self.pos[res]
        s=self.tas.pop()
        if self.vide():
            return res
        self.tas[0]=s
        self.pos[s]=0
        k=0
        fini=False
        while not fini:
            i=k
            if 2*k+1<len(self.tas) and self.prio[self.tas[2*k+1]]<self.prio[s]:
                i=2*k+1
            if 2*k+2<len(self.tas) and self.prio[self.tas[2*k+2]]<self.prio[self.tas[i]]:
                i=2*k+2
            if i==k:
                fini=True
            else:
                t=self.tas[i]
                self.tas[k],self.tas[i]=t,s
                self.pos[s]=i
                self.pos[t]=k
                k=i
        return res
    def modifie(self,s,p):
        self.prio[s]=p
        k=self.pos[s]
        while k>0 and self.prio[self.tas[k]]<self.prio[self.tas[(k-1)//2]]:
            self.tas[k],self.tas[(k-1)//2]=self.tas[(k-1)//2],self.tas[k]
            self.pos[self.tas[k]]=k
            self.pos[self.tas[(k-1)//2]]=(k-1)//2
            k=(k-1)//2
    def priorite(self,s):
        return self.prio[s]
    def distances(self):
        return self.prio
    def visu(self):
        chaine=''
        for s in self.tas:
            chaine+=str(s)+':'+str(self.prio[s])+' '
        return chaine

**Q1** Expliquer le rôle de chacun des `if` des lignes 21, 23 et 25.

**Q2** Quel est le rôle de la variable `fini` ?

**Q3** Commenter le code.

In [2]:
file=FileP('0123456')
print(file.visu())
file.modifie('6',0)
print(file.visu())

0:inf 1:inf 2:inf 3:inf 4:inf 5:inf 6:inf 
6:0 1:inf 0:inf 3:inf 4:inf 5:inf 2:inf 


In [3]:
file.modifie('3',4)
file.modifie('4',7)
file.modifie('5',12)
print(file.visu())

6:0 3:4 5:12 1:inf 4:7 0:inf 2:inf 


**Q4** Représenter les tas visualisés par chaque appel à `visu` sous forme d'arbres avec pour chaque noeud le nom du sommet et sa priorité.

In [4]:
file.distances()

{'0': inf, '1': inf, '2': inf, '3': 4, '4': 7, '5': 12, '6': 0}

On va travailler avec le graphe donné ci-dessous sous la forme d'un dictionnaire donnant des dictionnaires d'adjacence avec les poids.

In [5]:
g={'0':{'1':5,'2':7,'3':4,'4':2},'1':{'0':5,'4':2},'2':{'0':7,'3':9,'5':5},'3':{'0':4,'2':9,'4':3,'5':7,'6':4},
   '4':{'0':2,'1':2,'3':3,'6':7},'5':{'2':5,'3':7,'6':12},'6':{'3':4,'4':7,'5':12}}

**Q5** Dessiner le graphe.

On donne trois fonctions utiles pour accéder au graphe.

In [6]:
def sommets(g):
    return [s for s in g.keys()]
def voisins(g,s):
    return [v for v in g[s].keys()]
def poids(g,s,t):
    return g[s][t]

In [7]:
sommets(g)

['0', '1', '2', '3', '4', '5', '6']

In [8]:
voisins(g,'3')

['0', '2', '4', '5', '6']

In [9]:
poids(g,'3','5')

7

**Q6** Ecrire la fonction `dijkstra` et la tester.

In [20]:
from math import inf
def dijkstra(g,s):
    # créer une file de priorité à partir des sommets du graphe
    fp=FileP(sommets(g))
    # créer un dictionnaire des prédécesseurs avec comme clés les sommets et comme valeurs None
    d={s:inf for s in sommets(g)}
    # modifier la priorité du sommet de départ en zéro
    fp.modifie(s,0)
    # tant que la file de priorité n'est pas vide
    while not fp.vide(): 
        # extraire le minimum u
        u=fp.extraire()
        # pour chaque voisin v de u
        for v in voisins(g,u):
            # si la priorité de v est supérieure à la priorité de u + le poids de l'arête entre u et v
            if fp.priorite(v) > (fp.priorite(u) + poids(g,u,v)):
                # la priorité de v devient celle de u plus le poids de l'arête
                fp.modifie(v,fp.priorite(u) + poids(g,u,v))
                # le prédécesseur de v devient u
                d[v]=u
    # renvoyer le dictionnaire des priorités finales et le tableau des prédécesseurs
    return fp.distances(),d

**Q7** En repartant de `dijkstra`, écrire une fonction `prim`.

In [21]:
dijkstra(g,'6')

({'0': 8, '1': 9, '2': 13, '3': 4, '4': 7, '5': 11, '6': 0}, {'0': '3', '1': '4', '2': '3', '3': '6', '4': '6', '5': '3', '6': inf})